In [ ]:
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
#from matplotlib import pyplot as plt
import seaborn as sns
import plotly
import plotly_express as px
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer
from datetime import datetime

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [ ]:
airbnb_data = pd.read_csv('/Users/jaredsimpson/Desktop/351Datasets/airbnb_data.csv')

/var/folders/r5/s5xhk8xs0_qddhlqhv9p12mh0000gn/T/ipykernel_49901/2041984864.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_data = pd.read_csv('/Users/jaredsimpson/Desktop/351Datasets/airbnb_data.csv')


In [ ]:
# Calculate the percentage of missing values per column again
missing_data = airbnb_data.isnull().sum()

# Filter to show only columns with missing values and sort them
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

missing_data


host_neighbourhood              156208
neighbourhood_group_cleansed    149111
license                         145938
host_about                      109193
neighborhood_overview            98441
neighbourhood                    98441
host_location                    56161
host_response_time               56154
host_response_rate               56154
first_review                     48278
last_review                      48278
reviews_per_month                48278
listing_age_years                48278
recency_years                    48278
review_scores_value              48212
review_scores_location           48208
review_scores_checkin            48204
review_scores_communication      48189
review_scores_cleanliness        48184
review_scores_accuracy           48184
review_scores_variance           48169
average_review                   48155
review_scores_rating             48155
host_acceptance_rate             27216
price                            10811
has_availability         

In [ ]:
# Calculate the percentage of missing values per column again
missing_data = airbnb_data.isnull().mean() * 100

# Filter to show only columns with missing values and sort them
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

missing_data


host_neighbourhood              78.561226
neighbourhood_group_cleansed    74.991953
license                         73.396166
host_about                      54.916112
neighborhood_overview           49.508640
neighbourhood                   49.508640
host_location                   28.244885
host_response_time              28.241365
host_response_rate              28.241365
first_review                    24.280311
last_review                     24.280311
reviews_per_month               24.280311
listing_age_years               24.280311
recency_years                   24.280311
review_scores_value             24.247118
review_scores_location          24.245107
review_scores_checkin           24.243095
review_scores_communication     24.235551
review_scores_cleanliness       24.233036
review_scores_accuracy          24.233036
review_scores_variance          24.225492
average_review                  24.218451
review_scores_rating            24.218451
host_acceptance_rate            13

In [ ]:
missing_data[:20]

host_neighbourhood              78.561226
neighbourhood_group_cleansed    74.991953
license                         73.396166
host_about                      54.916112
neighborhood_overview           49.508640
neighbourhood                   49.508640
host_location                   28.244885
host_response_time              28.241365
host_response_rate              28.241365
first_review                    24.280311
last_review                     24.280311
reviews_per_month               24.280311
listing_age_years               24.280311
recency_years                   24.280311
review_scores_value             24.247118
review_scores_location          24.245107
review_scores_checkin           24.243095
review_scores_communication     24.235551
review_scores_cleanliness       24.233036
review_scores_accuracy          24.233036
dtype: float64

In [ ]:
missing_data[21:]

average_review               24.218451
review_scores_rating         24.218451
host_acceptance_rate         13.687662
price                         5.437144
has_availability              5.437144
bedrooms                      2.454284
beds                          1.312137
host_is_superhost             0.339476
baths                         0.229838
bathrooms_text                0.084995
host_verifications            0.004526
host_total_listings_count     0.004526
host_listings_count           0.004526
host_has_profile_pic          0.004526
host_thumbnail_url            0.004526
host_identity_verified        0.004526
host_duration_years           0.004526
host_name                     0.004526
host_since                    0.004526
host_picture_url              0.004526
dtype: float64

- Drop all rows where these columns values are NA:
    - host_duration_years
    - host_since
    - host_identity_verfied

- Transform (unless specified, binary 1 if its icnluded and 0 if not)
    - host_about
        - binary 1 if has about 0 if not
    - picture_url
    - host_picture_url
    - host_thumbnail_url
    - baths
        - has Half-bath, shared Half-bat, private half-bath
        - if any of these are included make it 0.5, otherwise if NAN make it 0


- Inputation (if not specified, use cluster mean on the column of the sub bullet point if given):
    - host_listings_count
        - input with the sum of that hosts id in the dataframe
    - beds
        - room_type
    - bedrooms
        - room_type

- Drop Columns:
    - host_neighbourhood
    - neighbourhood_group_cleansed
    - license
    - neighborhood_overview
    - neighbourhood
    - host_name
    - host_has_profile_pic
    - host_verifications
        - captured by verification_count
    - bathrooms_text (AFTER baths CHANGES)


- Other (leave for now)
    - host_is_superhost
    - has_availability
    - price
    - host_acceptance_rate
    - reviews
        - average_review
        - review_scores_rating
        - review_scores_value
        - review_scores_location
        - review_scores_checkin
        - review_scores_communication
        - review_scores_cleanliness    
        - review_scores_accuracy      
    - host_location
    - host_response_time
    - host_response_rate
    - first_review
    - last_review
    - reviews_per_month
    - listing_age_years
    - recency_years     

## Missing Vaules

### Step 1 (Broad/easier decisions)

- Drop rows where columns have very few missing values (and seem unimportant)
- Transform columns to binary
- Imputation
- Drop Columns

#### Drop Rows

In [ ]:
cols_to_drop_na = ['host_duration_years', 'host_since', 'host_identity_verified']
airbnb_data.dropna(subset=cols_to_drop_na, inplace=True)

#### Transform to Binary

In [ ]:
binary_columns = ['host_about', 'picture_url', 'host_picture_url', 'host_thumbnail_url']
for col in binary_columns:
    airbnb_data[col] = airbnb_data[col].notna().astype(int)

In [ ]:
bath_conditions = [
    'Half-bath', 'Shared half-bath', 'Private half-bath'
]
airbnb_data['baths'] = airbnb_data['bathrooms_text'].str.contains('|'.join(bath_conditions)).fillna(0).astype(int)
airbnb_data['baths'].replace({True: 0.5, False: 0}, inplace=True)

#### Imputation

In [ ]:
# Imputation for host_listings_count
host_listing_sums = airbnb_data.groupby('host_id')['host_listings_count'].transform('sum')
airbnb_data['host_listings_count'] = airbnb_data['host_listings_count'].fillna(host_listing_sums)

# Imputation for beds and bedrooms based on room_type
for col in ['beds', 'bedrooms']:
    airbnb_data[col] = airbnb_data.groupby('room_type')[col].transform(lambda x: x.fillna(x.mean()))

#### Drop Columns

In [ ]:
columns_to_drop = [
    'host_neighbourhood', 'neighbourhood_group_cleansed', 'license',
    'neighborhood_overview', 'neighbourhood', 'host_name',
    'host_has_profile_pic', 'host_verifications', 'bathrooms_text'
]
airbnb_data.drop(columns=columns_to_drop, inplace=True)

### Step 2 (Final Missing Value Handling)

In [ ]:
# Calculate the percentage of missing values per column again
missing_data = airbnb_data.isnull().mean() * 100

# Filter to show only columns with missing values and sort them
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

missing_data

host_location                  28.241637
host_response_rate             28.238117
host_response_time             28.238117
recency_years                  24.281410
listing_age_years              24.281410
reviews_per_month              24.281410
first_review                   24.281410
last_review                    24.281410
review_scores_value            24.248216
review_scores_location         24.246204
review_scores_checkin          24.244192
review_scores_communication    24.236648
review_scores_accuracy         24.234133
review_scores_cleanliness      24.234133
review_scores_variance         24.226589
review_scores_rating           24.219548
average_review                 24.219548
host_acceptance_rate           13.683755
has_availability                5.437390
price                           5.437390
host_is_superhost               0.339491
dtype: float64

- Drop Columns:
    - bathrooms_text
    - host_location
    - host_acceptance_rate

- Imputation
    - host_is_superhost
        - impute with 0 (assume not superhost if not listed)
    - review scores
        - cluster median based on property_type

Other:
    - Store all rows with price NA in seperate dataset called price_na (if wanted by others) and drop those rows from airbnb_data

#### host_location, host_response_rate, host_response_time

In [ ]:
# Replace missing 'host_location' with 'Unknown'
#airbnb_data['host_location'].fillna('Unknown', inplace=True)

# For 'host_response_rate' and 'host_response_time', replace missing with -1 and 'No Response'
#airbnb_data['host_response_rate'].fillna(-1, inplace=True)
#airbnb_data['host_response_time'].fillna('No Response', inplace=True)


In [ ]:
columns_to_drop = ['host_acceptance_rate', 'host_response_time', 'host_response_rate', 'host_location', 'first_review', 'last_review']
airbnb_data.drop(columns=columns_to_drop, inplace=True)

#### Time Related Features

In [ ]:
# For listings without reviews, it might be new or not active
max_recency = airbnb_data['recency_years'].max()
airbnb_data['recency_years'].fillna(max_recency, inplace=True)

max_listing_age = airbnb_data['listing_age_years'].max()
airbnb_data['listing_age_years'].fillna(max_listing_age, inplace=True)

# If 'reviews_per_month' is missing, assume 0 reviews in the past month
airbnb_data['reviews_per_month'].fillna(0, inplace=True)

/var/folders/r5/s5xhk8xs0_qddhlqhv9p12mh0000gn/T/ipykernel_49901/636542198.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  latest_date = pd.to_datetime(airbnb_data[['first_review', 'last_review']].max().max())


#### Missing Review Scores

In [ ]:
review_columns = ['review_scores_value', 'review_scores_location', 'review_scores_checkin',
                  'review_scores_communication', 'review_scores_accuracy',
                  'review_scores_cleanliness', 'review_scores_variance',
                  'review_scores_rating', 'average_review']

# Calculate the cluster median for each property type and review score
for column in review_columns:
    cluster_median = airbnb_data.groupby('property_type')[column].transform('median')
    airbnb_data[column] = airbnb_data[column].fillna(cluster_median)


/var/folders/r5/s5xhk8xs0_qddhlqhv9p12mh0000gn/T/ipykernel_49901/1620742568.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data[column] = airbnb_data[column].fillna(cluster_median)
/var/folders/r5/s5xhk8xs0_qddhlqhv9p12mh0000gn/T/ipykernel_49901/1620742568.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data[column] = airbnb_data[column].fillna(cluster_median)
/var/folders/r5/s5xhk8xs0_qddhlqhv9p12mh0000gn/T/ipykernel_49901/1620742568.py:9: SettingWithCopyWarning: 
A value is trying t

In [ ]:
# Some still have missing values, just impute the general median for the rest
median_scores = airbnb_data[['review_scores_value', 'review_scores_location',
                             'review_scores_checkin', 'review_scores_communication',
                             'review_scores_accuracy', 'review_scores_cleanliness',
                             'review_scores_variance', 'review_scores_rating',
                             'average_review']].median()

for col in median_scores.index:
    airbnb_data[col].fillna(median_scores[col], inplace=True)


/var/folders/r5/s5xhk8xs0_qddhlqhv9p12mh0000gn/T/ipykernel_49901/763683564.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data[col].fillna(median_scores[col], inplace=True)


#### Has-Availability

In [ ]:
# Replace missing 'has_availability' with the mode
#mode_availability = airbnb_data['has_availability'].mode()[0]
#airbnb_data['has_availability'].fillna(mode_availability, inplace=True)

In [ ]:
# Calculate the cluster mode for each 'host_is_superhost' status
mode_availability_by_superhost = airbnb_data.groupby('host_is_superhost')['has_availability'].apply(lambda x: x.mode()[0])

# Apply the mode to missing values in 'has_availability'
for superhost_status in mode_availability_by_superhost.index:
    condition = (airbnb_data['host_is_superhost'] == superhost_status) & (airbnb_data['has_availability'].isna())
    airbnb_data.loc[condition, 'has_availability'] = mode_availability_by_superhost[superhost_status]


#### Host-Is-Superhost

In [ ]:
# Replace missing 'host_is_superhost' with 0 (assuming not a superhost)
airbnb_data['host_is_superhost'].fillna(0, inplace=True)

#### Price

In [ ]:
# Store rows where price is NA
price_na = airbnb_data[airbnb_data['price'].isna()]

# Drop rows where price is NA from the original dataset
airbnb_data = airbnb_data.dropna(subset=['price'])

In [ ]:
# Calculate the percentage of missing values per column again
missing_data = airbnb_data.isnull().mean() * 100

# Filter to show only columns with missing values and sort them
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

missing_data

Series([], dtype: float64)

In [ ]:
airbnb_data.head()

,City,last_scraped,source,picture_url,host_id,host_url,host_since,host_about,host_is_superhost,host_thumbnail_url,host_picture_url,host_listings_count,host_total_listings_count,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,bedrooms,baths,verification_count,average_review,distance_to_centre,host_duration_years,listing_age_years,recency_years,review_scores_variance
0,Bergamo,2023-12-29,city scrape,1,21763382,https://www.airbnb.com/users/show/21763382,2014-09-25,1,f,1,1,88.0,210.0,t,Dossena,45.879800,9.697220,Entire rental unit,Entire home/apt,4,2.0,[],$142.00,1,1125,2,2,29,1125,2.0,1120.0,t,1,1,1,1,2023-12-29,7,1,0,4.43,4.71,4.86,5.00,4.57,4.86,4.57,t,15,15,0,0,0.28,1.0,0,2,4.714286,19.571726,9.448323,2.245038,0.547570,0.041095
1,Bergamo,2023-12-29,city scrape,1,392969278,https://www.airbnb.com/users/show/392969278,2021-03-17,1,f,1,1,30.0,35.0,t,Parzanica,45.736980,10.039873,Entire rental unit,Entire home/apt,4,2.0,[],$176.00,1,1125,3,3,1125,1125,3.0,1125.0,t,7,7,7,7,2023-12-29,13,4,0,4.85,4.69,4.85,4.77,4.69,4.85,4.62,t,13,13,0,0,0.63,1.0,0,3,4.760000,29.815869,2.973306,1.891855,0.484600,0.008967
2,Bergamo,2023-12-29,city scrape,1,517350267,https://www.airbnb.com/users/show/517350267,2023-05-30,0,f,1,1,1.0,1.0,t,Ranzanico,45.789980,9.950410,Entire rental unit,Entire home/apt,4,1.0,[],$160.00,2,29,2,2,1125,1125,2.0,1125.0,t,29,59,89,347,2023-12-29,6,6,0,4.83,5.00,5.00,5.00,5.00,5.00,4.83,f,1,1,0,0,1.13,1.0,0,2,4.951429,24.498594,0.772074,0.624230,0.451745,0.006881
3,Bergamo,2023-12-30,city scrape,1,2831947,https://www.airbnb.com/users/show/2831947,2012-07-04,1,t,1,1,10.0,22.0,t,Bergamo,45.687990,9.667020,Entire vacation home,Entire home/apt,6,5.0,[],$75.00,3,90,3,6,1125,1125,3.2,1125.0,t,15,39,60,300,2023-12-30,71,40,2,4.93,4.90,4.82,4.93,4.97,4.76,4.86,t,6,6,0,0,3.24,2.0,0,2,4.881429,2.053675,11.674196,1.982204,0.224504,0.005314
4,Bergamo,2023-12-30,city scrape,1,112088465,https://www.airbnb.com/users/show/112088465,2017-01-18,1,f,1,1,3.0,3.0,t,Bergamo,45.699067,9.675348,Entire rental unit,Entire home/apt,4,2.0,[],$100.00,2,30,2,2,30,30,2.0,30.0,t,13,22,52,327,2023-12-30,10,10,1,4.90,5.00,4.90,4.80,5.00,5.00,4.80,f,1,1,0,0,2.22,1.0,0,2,4.914286,1.437623,7.132101,0.553046,0.208077,0.008095


In [ ]:
airbnb_data.to_csv('/Users/jaredsimpson/Desktop/351Datasets/airbnb_data_mv.csv', index=False)

## Correct Data Types

In [ ]:
airbnb_data.dtypes[1:20]

last_scraped                 datetime64[ns]
source                               object
picture_url                           int64
host_id                               int64
host_url                             object
host_since                   datetime64[ns]
host_about                            int64
host_is_superhost                    object
host_thumbnail_url                    int64
host_picture_url                      int64
host_listings_count                 float64
host_total_listings_count           float64
host_identity_verified               object
neighbourhood_cleansed               object
latitude                            float64
longitude                           float64
property_type                        object
room_type                            object
accommodates                          int64
dtype: object

In [ ]:
airbnb_data['last_scraped'] = pd.to_datetime(airbnb_data['last_scraped'])
airbnb_data['host_since'] = pd.to_datetime(airbnb_data['host_since'])

/var/folders/r5/s5xhk8xs0_qddhlqhv9p12mh0000gn/T/ipykernel_49901/3987558791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data['last_scraped'] = pd.to_datetime(airbnb_data['last_scraped'])
/var/folders/r5/s5xhk8xs0_qddhlqhv9p12mh0000gn/T/ipykernel_49901/3987558791.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data['host_since'] = pd.to_datetime(airbnb_data['host_since'])


In [ ]:
airbnb_data.dtypes[21:40]

amenities                  object
price                      object
minimum_nights              int64
maximum_nights              int64
minimum_minimum_nights      int64
maximum_minimum_nights      int64
minimum_maximum_nights      int64
maximum_maximum_nights      int64
minimum_nights_avg_ntm    float64
maximum_nights_avg_ntm    float64
has_availability           object
availability_30             int64
availability_60             int64
availability_90             int64
availability_365            int64
calendar_last_scraped      object
number_of_reviews           int64
number_of_reviews_ltm       int64
number_of_reviews_l30d      int64
dtype: object

In [ ]:
# Remove the dollar sign and commas, then convert to float
airbnb_data['price'] = airbnb_data['price'].replace('[\$,]', '', regex=True).astype(float)

float64


/var/folders/r5/s5xhk8xs0_qddhlqhv9p12mh0000gn/T/ipykernel_49901/3403813305.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data['price'] = airbnb_data['price'].replace('[\$,]', '', regex=True).astype(float)


In [ ]:
airbnb_data['calendar_last_scraped'] = pd.to_datetime(airbnb_data['calendar_last_scraped'])

/var/folders/r5/s5xhk8xs0_qddhlqhv9p12mh0000gn/T/ipykernel_49901/3707141466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data['calendar_last_scraped'] = pd.to_datetime(airbnb_data['calendar_last_scraped'])


In [ ]:
airbnb_data.dtypes[41:]

review_scores_accuracy                          float64
review_scores_cleanliness                       float64
review_scores_checkin                           float64
review_scores_communication                     float64
review_scores_location                          float64
review_scores_value                             float64
instant_bookable                                 object
calculated_host_listings_count                    int64
calculated_host_listings_count_entire_homes       int64
calculated_host_listings_count_private_rooms      int64
calculated_host_listings_count_shared_rooms       int64
reviews_per_month                               float64
bedrooms                                        float64
baths                                             int64
verification_count                                int64
average_review                                  float64
distance_to_centre                              float64
host_duration_years                             

## Preliminary Feature Selection:

In [ ]:
airbnb_data.shape

(188016, 62)

In [ ]:
columns_to_drop = ['last_scraped', 'source', 'host_id']
airbnb_data.drop(columns=columns_to_drop, inplace=True)

In [ ]:
airbnb_data.columns[:30].tolist()

['City',
 'picture_url',
 'host_url',
 'host_since',
 'host_about',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_listings_count',
 'host_total_listings_count',
 'host_identity_verified',
 'neighbourhood_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'has_availability',
 'availability_30']

In [ ]:
airbnb_data.columns[31:].tolist()

['availability_90',
 'availability_365',
 'calendar_last_scraped',
 'number_of_reviews',
 'number_of_reviews_ltm',
 'number_of_reviews_l30d',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'instant_bookable',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms',
 'reviews_per_month',
 'bedrooms',
 'baths',
 'verification_count',
 'average_review',
 'distance_to_centre',
 'host_duration_years',
 'listing_age_years',
 'recency_years',
 'review_scores_variance']

In [ ]:
# Calculate correlation matrix for numerical features
correlation_matrix = airbnb_data.corr().abs()

# Select upper triangle of correlation matrix
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]

to_drop

/var/folders/r5/s5xhk8xs0_qddhlqhv9p12mh0000gn/T/ipykernel_49901/1965730595.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))


['recency_years']

In [ ]:
correlation_matrix

,picture_url,host_about,host_thumbnail_url,host_picture_url,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,bedrooms,baths,verification_count,average_review,distance_to_centre,host_duration_years,listing_age_years,recency_years,review_scores_variance
picture_url,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
host_about,NaN,1.000000,NaN,NaN,0.203745,0.187980,0.103486,0.100440,0.029002,0.011323,0.016387,0.004763,0.002687,0.004330,0.020980,0.000527,0.002147,0.012798,0.002144,0.082363,0.076870,0.075770,0.031229,0.165570,0.106775,0.069606,0.050346,0.036962,0.052515,0.047704,0.051895,0.009258,0.043950,0.162847,0.165575,0.099876,0.010378,0.072439,0.020947,0.006236,0.232354,0.043905,0.083199,0.359446,0.004677,0.104914,0.006185
host_thumbnail_url,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
host_picture_url,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
host_listings_count,NaN,0.203745,NaN,NaN,1.000000,0.936091,0.039745,0.109563,0.120941,0.080169,0.057493,0.022123,0.000487,0.016558,0.006770,0.001275,0.001631,0.008093,0.001632,0.016394,0.030982,0.038322,0.033502,0.083745,0.084908,0.052660,0.091396,0.085290,0.077055,0.085685,0.102387,0.045647,0.099981,0.720569,0.723479,0.574609,0.011387,0.089309,0.095351,0.008196,0.060102,0.108077,0.107019,0.013464,0.078315,0.126714,0.081218
host_total_listings_count,NaN,0.187980,NaN,NaN,0.936091,1.000000,0.047989,0.126122,0.117817,0.078117,0.043203,0.019929,0.000416,0.013639,0.007685,0.001142,0.001471,0.005632,0.001472,0.036404,0.050261,0.057323,0.017031,0.077497,0.080507,0.051129,0.078592,0.075316,0.064141,0.071311,0.088314,0.041484,0.087714,0.762980,0.762624,0.655365,0.038907,0.086116,0.091767,0.007291,0.046358,0.094075,0.112754,0.018629,0.076608,0.120191,0.076838
latitude,NaN,0.103486,NaN,NaN,0.039745,0.047989,1.000000,0.626386,0.153615,0.142422,0.018220,0.020796,0.002475,0.017509,0.025915,0.003041,0.003470,0.023794,0.003471,0.135916,0.108168,0.098632,0.205699,0.180271,0.229597,0.227196,0.022462,0.000338,0.041123,0.054596,0.041247,0.036575,0.049628,0.001698,0.004475,0.040266,0.023197,0.268850,0.145857,0.012126,0.107899,0.019137,0.678614,0.082111,0.152900,0.177858,0.023582
longitude,NaN,0.100440,NaN,NaN,0.109563,0.126122,0.626386,1.000000,0.165662,0.163718,0.008296,0.016597,0.001459,0.014562,0.018143,0.003619,0.004336,0.018158,0.004336,0.123027,0.087413,0.075499,0.189412,0.183169,0.238847,0.230141,0.027811,0.000744,0.049997,0.051316,0.039532,0.007999,0.045464,0.099207,0.095861,0.134246,0.022508,0.281046,0.145630,0.012983,0.093092,0.023130,0.652286,0.099797,0.156618,0.189005,0.024613
accommodates,NaN,0.029002,NaN,NaN,0.120941,0.117817,0.153615,0.165662,1.000000,0.795686,0.084121,0.005436,0.002167,0.003563,0.002312,0.003634,0.005921,0.000772,0.005920,0.001906,0.009654,0.012204,0.065339,0.068142,0.085602,0.066293,0.005909,0.030672,0.025304,0.002355,0.006704,0.021257,0.003971,0.076617,0.081174,0.001518,0.019349,0.106677

Drop all of the other "nights" features as it is captured by minimum_nights_avg_ntm and maximum_nights_avg_ntm

In [ ]:
columns_to_drop = ['host_total_listings_count', 'minimum_minimum_nights', 'maximum_minimum_nights', 'maximum_maximum_nights', 'minimum_nights', 'maximum_nights', 'minimum_maximum_nights']
airbnb_data.drop(columns=columns_to_drop, inplace=True)

In [ ]:
columns_to_drop = ['availability_30', 'availability_60', 'availability_365']
airbnb_data.drop(columns=columns_to_drop, inplace=True)

Maybe want to keep "number of reviews", but captured by reviews_per_month

In [ ]:
columns_to_drop = ['host_url', 'neighbourhood_cleansed', 'latitude', 'longitude',
                   'calculated_host_listings_count',
                   'calculated_host_listings_count_entire_homes',
                   'calculated_host_listings_count_private_rooms',
                   'calculated_host_listings_count_shared_rooms',
                   'number_of_reviews',
                   'number_of_reviews_ltm',
                   'number_of_reviews_l30d']
airbnb_data.drop(columns=columns_to_drop, inplace=True)

Drop all reviews but but average review score

In [ ]:
columns_to_drop = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
                   'review_scores_location', 'review_scores_value']
airbnb_data.drop(columns=columns_to_drop, inplace=True)

In [ ]:
columns_to_drop = ['host_since', 'minimum_maximum_nights', 'calendar_last_scraped']
airbnb_data.drop(columns=columns_to_drop, inplace=True)

In [ ]:
airbnb_data.head()

,City,picture_url,host_about,host_is_superhost,host_thumbnail_url,host_picture_url,host_listings_count,host_identity_verified,property_type,room_type,accommodates,beds,amenities,price,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_90,instant_bookable,reviews_per_month,bedrooms,baths,verification_count,average_review,distance_to_centre,host_duration_years,listing_age_years,recency_years,review_scores_variance
0,Bergamo,1,1,f,1,1,88.0,t,Entire rental unit,Entire home/apt,4,2.0,[],142.0,2.0,1120.0,t,1,t,0.28,1.0,0,2,4.714286,19.571726,9.448323,2.245038,0.547570,0.041095
1,Bergamo,1,1,f,1,1,30.0,t,Entire rental unit,Entire home/apt,4,2.0,[],176.0,3.0,1125.0,t,7,t,0.63,1.0,0,3,4.760000,29.815869,2.973306,1.891855,0.484600,0.008967
2,Bergamo,1,0,f,1,1,1.0,t,Entire rental unit,Entire home/apt,4,1.0,[],160.0,2.0,1125.0,t,89,f,1.13,1.0,0,2,4.951429,24.498594,0.772074,0.624230,0.451745,0.006881
3,Bergamo,1,1,t,1,1,10.0,t,Entire vacation home,Entire home/apt,6,5.0,[],75.0,3.2,1125.0,t,60,t,3.24,2.0,0,2,4.881429,2.053675,11.674196,1.982204,0.224504,0.005314
4,Bergamo,1,1,f,1,1,3.0,t,Entire rental unit,Entire home/apt,4,2.0,[],100.0,2.0,30.0,t,52,f,2.22,1.0,0,2,4.914286,1.437623,7.132101,0.553046,0.208077,0.008095


In [ ]:
airbnb_data.shape

(188016, 29)

In [ ]:
unique_values_count = len(airbnb_data['property_type'].unique())

print("Number of unique values in the column:", unique_values_count)

Number of unique values in the column: 127


## Encode Categorical Features

In [ ]:
airbnb_data.columns

Index(['City', 'picture_url', 'host_about', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_listings_count',
       'host_identity_verified', 'property_type', 'room_type', 'accommodates',
       'beds', 'amenities', 'price', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'has_availability', 'availability_90',
       'instant_bookable', 'reviews_per_month', 'bedrooms', 'baths',
       'verification_count', 'average_review', 'distance_to_centre',
       'host_duration_years', 'listing_age_years', 'recency_years',
       'review_scores_variance'],
      dtype='object')

In [ ]:
airbnb_data.select_dtypes(include=['object']).columns.tolist()

['City', 'property_type', 'room_type', 'amenities']

In [ ]:
# Assuming the DataFrame is named airbnb_data
binary_columns = ['host_is_superhost', 'host_identity_verified', 'has_availability', 'instant_bookable']

# Map 't' to 1 and 'f' to 0
for col in binary_columns:
    airbnb_data[col] = airbnb_data[col].map({'t': 1, 'f': 0})

In [ ]:
# Assuming `airbnb_data` is your DataFrame and `price` is your target variable.
for cat_col in ['City', 'room_type']:
    # Group by the category and calculate median price
    median_price_by_cat = airbnb_data.groupby(cat_col)['price'].median().sort_values()

    # Create a mapping from category to median price
    median_price_mapping = {k: i for i, k in enumerate(median_price_by_cat.index, 0)}

    # Map the categories to their encoded labels
    airbnb_data[cat_col + '_encoded'] = airbnb_data[cat_col].map(median_price_mapping)

columns_to_drop = ['City', 'room_type']
airbnb_data.drop(columns=columns_to_drop, inplace=True)


In [ ]:
airbnb_data.head()

,picture_url,host_about,host_is_superhost,host_thumbnail_url,host_picture_url,host_listings_count,host_identity_verified,accommodates,beds,amenities,price,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_90,instant_bookable,reviews_per_month,bedrooms,baths,verification_count,average_review,distance_to_centre,host_duration_years,listing_age_years,recency_years,review_scores_variance,City_encoded,room_type_encoded
0,1,1,0.0,1,1,88.0,1,4,2.0,[],142.0,2.0,1120.0,1,1,1,0.28,1.0,0,2,4.714286,19.571726,9.448323,2.245038,0.547570,0.041095,4,2
1,1,1,0.0,1,1,30.0,1,4,2.0,[],176.0,3.0,1125.0,1,7,1,0.63,1.0,0,3,4.760000,29.815869,2.973306,1.891855,0.484600,0.008967,4,2
2,1,0,0.0,1,1,1.0,1,4,1.0,[],160.0,2.0,1125.0,1,89,0,1.13,1.0,0,2,4.951429,24.498594,0.772074,0.624230,0.451745,0.006881,4,2
3,1,1,1.0,1,1,10.0,1,6,5.0,[],75.0,3.2,1125.0,1,60,1,3.24,2.0,0,2,4.881429,2.053675,11.674196,1.982204,0.224504,0.005314,4,2
4,1,1,0.0,1,1,3.0,1,4,2.0,[],100.0,2.0,30.0,1,52,0,2.22,1.0,0,2,4.914286,1.437623,7.132101,0.553046,0.208077,0.008095,4,2


## Final Feature Engineering/Reduction

#### Random Forest Feature Reduction

In [ ]:
columns_to_drop = ['property_type','amenities']
airbnb_data.drop(columns=columns_to_drop, inplace=True)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


# Assuming airbnb_data is your DataFrame and 'price' is the target variable
X = airbnb_data.drop('price', axis=1)  # Features
y = airbnb_data['price']               # Target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model on the training data
rf_model.fit(X_train, y_train)

# Get the feature importances
importances = rf_model.feature_importances_

# Sort the features by their importance
sorted_indices = np.argsort(importances)[::-1]

# List features and their importances
for idx in sorted_indices:
    print(f"{X_train.columns[idx]}: {importances[idx]}")


distance_to_centre: 0.22094478144994242
host_duration_years: 0.10556686914155926
listing_age_years: 0.07891189732853188
host_listings_count: 0.07861689296698597
average_review: 0.07242617376077576
recency_years: 0.05931907859112832
minimum_nights_avg_ntm: 0.05255662731855894
availability_90: 0.04634114459581382
maximum_nights_avg_ntm: 0.04148213917703058
review_scores_variance: 0.03864408103052978
beds: 0.036877930864644845
reviews_per_month: 0.035540478221500214
accommodates: 0.026997815062477542
City_encoded: 0.025957381219336686
bedrooms: 0.020900422471230404
host_about: 0.016507184167279917
host_identity_verified: 0.012676847333634769
verification_count: 0.011319717564750772
instant_bookable: 0.008241060810944067
room_type_encoded: 0.0058878390027206065
host_is_superhost: 0.0040950276732465585
has_availability: 0.0001806792572269024
baths: 7.93099014979628e-06
host_picture_url: 0.0
host_thumbnail_url: 0.0
picture_url: 0.0


In [ ]:
columns_to_drop = ['host_picture_url','host_thumbnail_url', 'picture_url']
airbnb_data.drop(columns=columns_to_drop, inplace=True)

In [ ]:
airbnb_data.head()

,host_about,host_is_superhost,host_listings_count,host_identity_verified,accommodates,beds,price,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_90,instant_bookable,reviews_per_month,bedrooms,baths,verification_count,average_review,distance_to_centre,host_duration_years,listing_age_years,recency_years,review_scores_variance,City_encoded,room_type_encoded
0,1,0.0,88.0,1,4,2.0,142.0,2.0,1120.0,1,1,1,0.28,1.0,0,2,4.714286,19.571726,9.448323,2.245038,0.547570,0.041095,4,2
1,1,0.0,30.0,1,4,2.0,176.0,3.0,1125.0,1,7,1,0.63,1.0,0,3,4.760000,29.815869,2.973306,1.891855,0.484600,0.008967,4,2
2,0,0.0,1.0,1,4,1.0,160.0,2.0,1125.0,1,89,0,1.13,1.0,0,2,4.951429,24.498594,0.772074,0.624230,0.451745,0.006881,4,2
3,1,1.0,10.0,1,6,5.0,75.0,3.2,1125.0,1,60,1,3.24,2.0,0,2,4.881429,2.053675,11.674196,1.982204,0.224504,0.005314,4,2
4,1,0.0,3.0,1,4,2.0,100.0,2.0,30.0,1,52,0,2.22,1.0,0,2,4.914286,1.437623,7.132101,0.553046,0.208077,0.008095,4,2


In [ ]:
airbnb_data.to_csv('/Users/jaredsimpson/Desktop/351Datasets/airbnb_data_v2.csv', index=False)